In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install -U imbalanced-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 2.1 MB/s eta 0:00:00


In [3]:
!pip install scikit-learn==1.2.2

In [4]:
import pandas as pd
import random
filename = '/content/drive/MyDrive/output.csv'
data = pd.read_csv(filename, encoding='utf-8', header=None)

In [5]:
len(data)

384705

In [6]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,9496,9497,9498,9499,9500,9501,9502,9503,9504,9505
0,NaN,0,1,2,3,4,5,6,7,8,...,9495,9496,9497,9498,9499,9500.0,9501.0,9502.0,9503.0,Label
1,0.0,0004e68a0c0b4818c4e52d4f414d9dd2,49,0,0,1,0,0,0,6,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector
2,1.0,00b4a1d896a0513fc1ffe333e1c313f5,75,0,0,14,6,3,3,23,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector
3,2.0,00c525d5599ab0212e15cda186234260,1,0,0,144,1,3,0,17,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector
4,3.0,013377a94a52afc0fee128086d341e17,87,1,0,9,1,0,0,0,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector


In [7]:
data = data.drop(columns=[1])
data.head()

,0,2,3,4,5,6,7,8,9,10,...,9496,9497,9498,9499,9500,9501,9502,9503,9504,9505
0,NaN,1,2,3,4,5,6,7,8,9,...,9495,9496,9497,9498,9499,9500.0,9501.0,9502.0,9503.0,Label
1,0.0,49,0,0,1,0,0,0,6,1,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector
2,1.0,75,0,0,14,6,3,3,23,11,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector
3,2.0,1,0,0,144,1,3,0,17,6,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector
4,3.0,87,1,0,9,1,0,0,0,1,...,0,0,0,0,0,0.0,0.0,0.0,0.0,FileInfector


In [8]:
data.isna().sum()

0        1
2        0
3        0
4        0
5        0
        ..
9501    70
9502    70
9503    70
9504    72
9505     0
Length: 9505, dtype: int64

In [9]:
data.dropna(inplace=True)
data.isna().sum()

0       0
2       0
3       0
4       0
5       0
       ..
9501    0
9502    0
9503    0
9504    0
9505    0
Length: 9505, dtype: int64

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import joblib
import numpy as np
from sklearn.metrics import accuracy_score

In [11]:
def Feature_Selection(X_Data, Y_Data):
    print("Feature Selection")
    model = RandomForestClassifier(n_estimators=500, random_state=0, n_jobs=1, max_depth=25)
    model.fit(X_Data, Y_Data.values.ravel())
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    print(f"Number of features before selection: {X_Data.shape[1]}")
    threshold = 0.001
    selected_features = [X_Data.columns[i] for i in range(X_Data.shape[1]) if importances[indices[i]] > threshold]
    print(f"Number of features after selection: {len(selected_features)}")
    print("Selected Features:", selected_features)
    X_Data_Best = X_Data[selected_features].astype('uint8')
    return X_Data_Best

In [12]:
# def Label_Encoding(Y_Data):
#     print("Label Encoding")
#     le = LabelEncoder()
#     Y_Data_Final = le.fit_transform(Y_Data)
#     num_classes = len(le.classes_)
#     Y_Data_Final = to_categorical(Y_Data_Final, num_classes=num_classes)
#     return Y_Data_Final
def Label_Encoding(Y_Data):
    print("Label Encoding")
    le = LabelEncoder()
    Y_Data_Final = le.fit_transform(Y_Data)
    return Y_Data_Final

In [13]:
from imblearn.over_sampling import SMOTE

def MyModel(X_Data_Final, Y_Data_Final):
    print("Model")
    X_tr, X_te, Y_tr, Y_te = train_test_split(X_Data_Final, Y_Data_Final, test_size=0.20, random_state=42)
    smote = SMOTE(random_state=42)
    X_tr, Y_tr = smote.fit_resample(X_tr, Y_tr)
    model = RandomForestClassifier(n_estimators=1000, random_state=42, max_depth=30, min_samples_split=5, class_weight='balanced')
    model.fit(X_tr, Y_tr)
    joblib.dump(model, 'SavedWeights.joblib')
    print("Weights Saved.")
    Y_Predict = model.predict(X_te)
    print("Model Accuracy: ", accuracy_score(Y_te, Y_Predict))
    print(classification_report(Y_te, Y_Predict, digits=2))

In [14]:
def main():
    print("Python main function")
    Y_Data = data.iloc[:, -1]
    X_Data = data.iloc[:, :-1]
    X_Data_Final = Feature_Selection(X_Data, Y_Data)
    Y_Data_Final = Label_Encoding(Y_Data)
    MyModel(X_Data_Final, Y_Data_Final)

In [15]:
if __name__ == '__main__':
    main()

Python main function
Feature Selection
Number of features before selection: 9504
Number of features after selection: 140
Selected Features: [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140]
Label Encoding
Model
Weights Saved.
Model Accuracy:  0.9141133802176089
              precision    recall  f1-score   support

           0       0.93      0.87      0.90      9437
           1       0.54      0.74      0.62       292
           2       0.73      0.8

In [ ]:
# 52 min --> feature selection